In [30]:
import sqlite3
import pandas as pd
import numpy as np

In [31]:
db = sqlite3.connect("db.sqlite3")

## Tables of Use

### Data to Feed into Model
1. gui_alwaysregex to get all the always patterns
2. gui_sentenceannotation to get labels
3. gui_sentence to get sentences

### EDA Purposes
1. gui_sentencealwaysregex to see which always expressions get flagged most often
2. gui_seedregex to get list of matched phrases
2. gui_sentenceseedregex to see which expressions in general get flagged most often

## Extracting Data for Model

In [32]:
always_patterns = pd.read_sql_query("Select * from gui_alwaysregex", db)
always_patterns

,id,Pattern,Annotation,CreatedBy_id
0,1,Given his cognitive deficits,Y,1
1,2,Dementia Mother,T,1
2,3,challenges with cognitive deficits,Y,1
3,4,on Aricept,Y,1
4,5,patient has dementia\r,Y,1
5,6,on Namenda\r,Y,1
6,8,"DIAGNOSES:(\b[A-Za-z0-9.()]*\b){0,20}\s*Memory...",Y,1
7,9,DIAGNOSES.*Memory loss,Y,1
8,10,Neurontin.*cause of.*cognitive dysfunction,T,1
9,11,cognitive\s*impairment\s*only\s*resolved,N,1


In [33]:
labels = pd.read_sql_query("Select * from gui_sentenceannotation", db)
#labels = labels.sort_values(["Sentence_id"])
labels

,id,Label,Annotator_id,Sentence_id
0,1,YES,1,20359
1,2,NO,1,2770
2,3,NTR,1,14691
3,4,YES,1,12033
4,5,NO,1,13803
5,6,NTR,1,12259
6,7,YES,1,5652
7,8,YES,1,6225
8,9,YES,1,14664
9,10,YES,1,17750


In [34]:
sentences = pd.read_sql_query("Select * from gui_sentence", db)
sentences

,id,Contents,Note_id
0,1,Visit Other Routine general medical examinatio...,1
1,2,• Doxepin Other (See Comments) vivid dreams; •...,2
2,3,accident moving next year due to extra bedroom...,3
3,4,techniques discuss again correlations long ter...,3
4,5,"benzo rxs 4/98, 9/97, 5/04 Family history: -mo...",3
...,...,...,...
23711,23712,and Coronary artery disease. who presents with...,9985
23712,23713,or recreational drug use. Family History: Moth...,9986
23713,23714,Family History Problem Relation Age of Onset •...,9987
23714,23715,Age of Onset • Rheumatoid arthritis Mother Rhe...,9987


## Constructing Model Matrix

In [35]:
note = pd.read_sql_query("Select * from gui_note", db)
note

,id,PatientEncounterID,NoteID,Date,PatientID
0,1,3115772095.0,1000021019,2016-04-28,Z6708893
1,2,3116814463.0,1000431290,2016-04-28,Z7766129
2,3,3107134257.0,1001733204,2016-04-29,Z9132953
3,4,,1003927065,2016-05-01,Z14342457
4,5,3103868830.0,1004813560,2016-05-02,Z9579170
...,...,...,...,...,...
9983,9984,3114479209.0,970688016,2016-04-10,Z14342457
9984,9985,3115574139.0,978113347,2016-04-14,Z9833197
9985,9986,3110202540.0,978789346,2016-04-14,Z10910426
9986,9987,3076173424.0,993123218,2016-04-25,Z8133252


In [36]:
print(note.loc[note['id'] == 8481]['PatientID'])

8480    Z8399761
Name: PatientID, dtype: object


In [43]:
#getting sentence ids
sentence_id = labels["Sentence_id"].to_list()

# defining model matrix and setting dimensions
model_matrix = pd.DataFrame(columns = ["patient_id", "sequence", "label"])
model_matrix["patient_id"] = [0] * int(0.85 * len(sentence_id))
model_matrix["sequence"] = [" "] * int(0.85 * len(sentence_id))
model_matrix["label"] = [" "] * int(0.85 * len(sentence_id))

#extracting necessary data
for i in range(int(0.85 * len(labels.axes[0]))):
    curr_note_id = sentences[sentences['id'] == sentence_id[i]]['Note_id'].values
    curr_patient_id = note.loc[note['id'] == curr_note_id[0]]['PatientID'].values
    curr_seq = sentences[sentences['Note_id'] == curr_note_id[0]]['Contents'].values
    curr_label = labels[labels["Sentence_id"] == sentence_id[i]]['Label'].values
    model_matrix.iloc[i] = (curr_patient_id[0], curr_seq[0], curr_label[0])

#converting yes,no,ntr labels to numerical equivalents
mappings = {"NO" : 1, "NTR" : 2, "YES" : 3}
model_matrix.label = [mappings[item] for item in model_matrix.label]

# exporting data to csv's
model_matrix.to_csv("input.csv")

# yes = model_matrix[model_matrix['label'] == 3]
# neither = model_matrix[model_matrix['label'] == 2]
# no = model_matrix[model_matrix['label'] == 1]

# yes = yes.to_csv("yes.csv")
# neither = neither.to_csv("neither.csv")
# no = no.to_csv("no.csv")

model_matrix

,patient_id,sequence,label
0,Z8399761,"(unclear history), venous insufficiency, PVD, ...",3
1,Z11957588,period. It apparently cuts car accident rates ...,1
2,Z10809276,(P) 5 Hearing: no concerns Vision: needs updat...,2
3,Z8539944,"remission, s/p 1 suicide attempt, 3 IPLOC admi...",3
4,Z12696803,should be pursued at that point in time. He an...,1
5,Z7552077,since the hospital visit. The patient is accom...,2
6,Z11702635,71 y.o. female with has a past medical history...,3
7,Z15548862,in the Memory Diagnostic Clinic at McLean Hosp...,3
8,Z7648831,on back wound with significant output per RN. ...,3
9,Z7701777,lobes bilaterally and left parietal lobe. Thes...,3


In [47]:
# defining test model matrix and setting dimensions
test_model_matrix = pd.DataFrame(columns = ["patient_id", "sequence", "label"])
test_model_matrix["patient_id"] = [0] * int(0.15 * len(sentence_id))
test_model_matrix["sequence"] = [" "] * int(0.15 * len(sentence_id))
test_model_matrix["label"] = [" "] * int(0.15 * len(sentence_id))

idx = int(0.85 * len(labels.axes[0])) + 1
i = 0
while (idx < len(labels.axes[0])):
    curr_note_id = sentences[sentences['id'] == sentence_id[idx]]['Note_id'].values
    curr_patient_id = note.loc[note['id'] == curr_note_id[0]]['PatientID'].values
    curr_seq = sentences[sentences['Note_id'] == curr_note_id[0]]['Contents'].values
    curr_label = labels[labels["Sentence_id"] == sentence_id[idx]]['Label'].values
    test_model_matrix.iloc[i] = (curr_patient_id[0], curr_seq[0], curr_label[0])
    idx += 1
    i += 1

#converting yes,no,ntr labels to numerical equivalents
mappings = {"NO" : 1, "NTR" : 2, "YES" : 3}
test_model_matrix.label = [mappings[item] for item in test_model_matrix.label]

# exporting data to csv's
test_model_matrix.to_csv("input_test.csv")

test_model_matrix

,patient_id,sequence,label
0,Z7766129,(akathisia) diphenhydramine (urinary retention...,2
1,Z12274682,is notable for maternal grandmother with epile...,1
2,Z10490939,Bundle branch block Cataract Nonexudative age-...,3


## EDA

In [42]:
regex_phrases = pd.read_sql_query("Select * from gui_seedregex", db)
phrase_matches = pd.read_sql_query("Select * from gui_sentenceseedregex", db)
freq = phrase_matches['SeedRegex_id'].value_counts().to_dict()
counter = 1
for key, value in freq.items():
    print("Match #", counter)
    print("Id: ", key)
    print("Pattern:", regex_phrases[regex_phrases['id'] == key]['Pattern'].values)
    print("Count: ", value, "\n")
    counter += 1

Match # 1
Id:  3
Pattern: ['dementia']
Count:  4135 

Match # 2
Id:  5
Pattern: ['Dementia']
Count:  3521 

Match # 3
Id:  7
Pattern: ["Alzheimer's disease"]
Count:  3311 

Match # 4
Id:  15
Pattern: ['cognitive impairment']
Count:  1907 

Match # 5
Id:  11
Pattern: ['memory loss']
Count:  1662 

Match # 6
Id:  13
Pattern: ['cognitive decline']
Count:  1211 

Match # 7
Id:  2
Pattern: ['memory problem']
Count:  906 

Match # 8
Id:  63
Pattern: ['Memory loss']
Count:  577 

Match # 9
Id:  26
Pattern: ['cognitive deficits']
Count:  543 

Match # 10
Id:  16
Pattern: ['Mild cognitive impairment']
Count:  499 

Match # 11
Id:  24
Pattern: ['memory impairment']
Count:  494 

Match # 12
Id:  121
Pattern: ['neurocognitive disorder']
Count:  400 

Match # 13
Id:  38
Pattern: ["other Alzheimer's disease"]
Count:  385 

Match # 14
Id:  12
Pattern: ['vascular dementia']
Count:  378 

Match # 15
Id:  19
Pattern: ["Alzheimer's dementia"]
Count:  372 

Match # 16
Id:  22
Pattern: ['memory deficit']
C

Id:  39
Pattern: ['Memory disorder']
Count:  9 

Match # 148
Id:  103
Pattern: ['Impaired insight']
Count:  9 

Match # 149
Id:  242
Pattern: ['Minor neurocognitive disorder']
Count:  9 

Match # 150
Id:  211
Pattern: ['ALZHEIMERS']
Count:  8 

Match # 151
Id:  186
Pattern: ['MEMORY DEFICIT']
Count:  8 

Match # 152
Id:  162
Pattern: ['impaired orientation']
Count:  8 

Match # 153
Id:  204
Pattern: ["early onset alzheimer's dementia"]
Count:  8 

Match # 154
Id:  230
Pattern: ['COGNITIVE DECLINE']
Count:  8 

Match # 155
Id:  198
Pattern: ['Mixed dementia']
Count:  8 

Match # 156
Id:  182
Pattern: ['primary progressive aphasia']
Count:  8 

Match # 157
Id:  165
Pattern: ['History of Dementia']
Count:  8 

Match # 158
Id:  35
Pattern: ['Dementia with Lewy Bodies']
Count:  8 

Match # 159
Id:  179
Pattern: ['Impaired memory']
Count:  7 

Match # 160
Id:  205
Pattern: ['Advanced Dementia']
Count:  7 

Match # 161
Id:  222
Pattern: ['Alzheimers disease']
Count:  7 

Match # 162
Id:  223


In [44]:
# Printing out frequencies of always pattern matches 
always_pattern_matches = pd.read_sql_query("Select * from gui_sentencealwaysregex", db)
freq = always_pattern_matches['AlwaysRegex_id'].value_counts().to_dict()
counter = 1
for key, value in freq.items():
    print("Match #", counter)
    print("Id: ", key)
    print("Pattern:", always_patterns[always_patterns['id'] == key]['Pattern'].values)
    print("Count: ", value, "\n")
    counter += 1

Match # 1
Id:  2
Pattern: ['Dementia Mother']
Count:  643 

Match # 2
Id:  4
Pattern: ['on Aricept']
Count:  41 

Match # 3
Id:  13
Pattern: ['Global CDR was 0.5']
Count:  29 

Match # 4
Id:  10
Pattern: ['Neurontin.*cause of.*cognitive dysfunction']
Count:  28 

Match # 5
Id:  9
Pattern: ['DIAGNOSES.*Memory loss']
Count:  27 

Match # 6
Id:  16
Pattern: ['patient([^\\.]*)diagnosis([^\\.]*)• Frontotemporal dementia']
Count:  27 

Match # 7
Id:  17
Pattern: ['contributing([^\\.]*)memory([^\\.]*)impairment']
Count:  27 

Match # 8
Id:  12
Pattern: ['cognitive\\s*impairment\\s*only,\\s*resolved']
Count:  26 

Match # 9
Id:  3
Pattern: ['challenges with cognitive deficits']
Count:  10 

Match # 10
Id:  7
Pattern: []
Count:  9 

Match # 11
Id:  14
Pattern: ['(he|she|they) (is) (co-followed(\\s*)) (by the (cognitive disorders|cognitive disorder|cognitive impairment) clinic)']
Count:  4 

Match # 12
Id:  15
Pattern: ['continue([^\\.]*)difficulty([^\\.]*)memory']
Count:  2 

Match # 13
Id:  1
